In [1]:
from collections import Counter
import matplotlib.pyplot as plt
from pysankey2 import Sankey
from scipy import sparse
from scipy.sparse import hstack, vstack, coo_matrix
from scipy.io import mmread
from sklearn import metrics
import math
import time
import anndata
import torch
import numpy as np
import pandas as pd
import scanpy as sc
import random
import os
from warnings import filterwarnings
import scipy.sparse as sp
from operator import itemgetter
filterwarnings("ignore")
import json, os
import math, copy, time
import numpy as np
from collections import defaultdict
import pyHGT2
from pyHGT2.data import *
from pyHGT2.utils import *
from pyHGT2.model1 import *
from pyHGT2.conv import *
from sklearn import metrics

In [2]:
from collections import Counter
import matplotlib.pyplot as plt
from pysankey2 import Sankey
from scipy.sparse import hstack, vstack, coo_matrix
from scipy.io import mmread
from sklearn import metrics
import math
import time
import anndata
import torch
import numpy as np
import pandas as pd
import scanpy as sc
import random
import os
from scipy import sparse
from scipy.io import mmread
from scipy.sparse import hstack, vstack, coo_matrix

In [3]:
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)
np.random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

#### Sim-PBMC 1-5

In [4]:
def sub_anndatas(index,ad1,dim1,ad2,dim2,ad3):
    def sub_ad(ad,id,dim):
        if dim == 0:
            ad = ad[id,:]
        elif dim == 1:
            ad = ad[:,id]
        return(ad)

    # sub sample
    ad1 = sub_ad(ad1,index,dim1)
    ad2 = sub_ad(ad2,index,dim2)

    # delete feature1 and feature2 not in sample
    index1 = np.where(np.array(ad1.X.sum(dim1)).squeeze()>0)[0]
    index2 = np.where(np.array(ad2.X.sum(dim2)).squeeze()>0)[0]
    ad3_new = sub_ad(sub_ad(ad3,index1,0),index2,1)

    # delete feature1 and feature2 not link
    index1 = index1[np.where(np.array(ad3_new.X.sum(1)).squeeze()>0)[0]]
    index2 = index2[np.where(np.array(ad3_new.X.sum(0)).squeeze()>0)[0]]

    ad1 = sub_ad(ad1,index1,1-dim1)
    ad2 = sub_ad(ad2,index2,1-dim2)
    ad3 = sub_ad(sub_ad(ad3,index1,0),index2,1)

    #delete sample don't in features
    index = np.where((np.array(ad1.X.sum(1-dim1)).squeeze()>0) & (np.array(ad2.X.sum(1-dim2)).squeeze()>0))[0]
    ad1 = sub_ad(ad1,index,dim1)
    ad2 = sub_ad(ad2,index,dim2)
    return (ad1,ad2,ad3,index1,index2,index)

In [34]:
os.chdir('/fs/ess/PCON0022/dmt/data/dataset/data1/')
cell_type = pd.read_csv('cell_type.csv', index_col=0)

gene_peak = anndata.read_mtx('Gene_Peak.mtx')
gene_cell = anndata.read_mtx('Gene_Cell.mtx')
peak_cell = anndata.read_mtx('Peak_Cell.mtx')
gene_names = pd.read_csv('Gene_names.tsv', sep='\t', header=None)
cell_names = pd.read_csv('Cell_names.tsv', sep='\t', header=None)
peak_names = pd.read_csv('Peak_names.tsv', sep='\t', header=None)
peak_cell.obs_names = peak_names[0]
peak_cell.var_names = cell_names[0]
gene_cell.obs_names = gene_names[0]
gene_cell.var_names = cell_names[0]
gene_peak.obs_names = gene_names[0]
gene_peak.var_names = peak_names[0]

#Gene_Cell, Peak_Cell, Gene_Peak = gene_cell,peak_cell,gene_peak
true_label = cell_type.loc[cell_names[0].tolist(), :]['cell_type']
cell_type['number'] = 1
cell_count = cell_type.groupby('cell_type').count()
cell_count =cell_count.sort_values(by='number',ascending=True, inplace=False)
rare_name = cell_count.iloc[0:3].index
ordinary_name = cell_count.iloc[-3:].index

In [35]:
cell_names = pd.read_csv('Cell_names.tsv', sep='\t', header=None)
true_label = cell_type.loc[cell_names[0].tolist(), :]['cell_type']
true_label

TAGTTGTCACCCTCAC-1-s1d1       Naive CD20+ B
CTATGGCCATAACGGG-1-s1d1          CD14+ Mono
CCGCACACAGGTTAAA-1-s1d1              CD8+ T
TCATTTGGTAATGGAA-1-s1d1              CD8+ T
ACCACATAGGTGTCCA-1-s1d1          CD16+ Mono
                                 ...       
GGTTTCTGTCGACTAA-3-s1d3          Normoblast
GGATGTAAGGATGATG-3-s1d3    CD4+ T activated
ATCTATGAGATGCCTG-3-s1d3        CD4+ T naive
CGCATTACACCAGGTT-3-s1d3                  NK
AGCTAGTTCAATAGCC-3-s1d3        CD8+ T naive
Name: cell_type, Length: 17243, dtype: object

In [36]:
cell_type['number'] = 1
cell_count = cell_type.groupby('cell_type').count()
cell_count =cell_count.sort_values(by='number',ascending=True, inplace=False)
rare_name = cell_count.iloc[0:3].index
ordinary_name = cell_count.iloc[-2:].index
rare_type = random.choice(rare_name)
ordinary_type = random.sample(list(ordinary_name),2)
cell_count

,number
cell_type,
ID2-hi myeloid prog,108
Plasma cell,379
ILC,835
cDC2,859
MK/E prog,884
CD8+ T naive,1012
HSC,1072
pDC,1191
G/M prog,1203


#### Sim-PBMC 1

In [11]:
# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
# rare_type = random.sample(list(rare_name),1)
# ordinary_type = random.sample(list(ordinary_name),1)
rare_type = ['Plasma cell']
ordinary_type = ['CD8+ T']
type_choice = ordinary_type + rare_type
total_num = 2
filetype_name = 'Ordinary1_Rare1'

for file_num in tqdm(range(50)):
    name_list =type_choice
    l = list()
    num_list = list()

    for i in range(0,total_num):
        if total_num == 2:
            if i == 0:
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),490)
                num_list = num_list+['490']
            else :
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),10)
                num_list = num_list+['10']
        else:
            if i == 0:
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),480)
                num_list = num_list+['480']
            else:
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),10)
                num_list = num_list+['10']
    l = np.sort(l)
    label = true_label[l]
    # file_name
    file_name = ''
    for i in range(total_num):
        if i <total_num-1:
            file_name = file_name +str(name_list[i])+str(num_list[i])+'_'
        else:
            file_name = file_name +str(name_list[i])+str(num_list[i])

    Gene_Cell, Peak_Cell, Gene_Peak, gene_index, peak_index, cell_index = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    GP_matrix = Gene_Peak.X
    
    sub_gene_name = list(gene_names[0][gene_index])
    sub_peak_name = list(peak_names[0][peak_index])
    
    # python_input
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA_'+file_name+'.txt', np.array([]))
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/label500.npy',label)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA'+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/ATAC'+'.npz', ATAC_matrix)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/Gene_Peak_'+'.npz', GP_matrix)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/gene_name.npy',sub_gene_name)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/peak_name.npy',sub_peak_name)
    
    ## R_input
    site1_GEX = np.array(RNA_matrix.transpose().todense(),dtype='int64')
    site1_ATAC = np.array(ATAC_matrix.transpose().todense(),dtype='int64')
    site1_label = label
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_GEX.txt',site1_GEX,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_ATAC.txt',site1_ATAC,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_label.txt',site1_label,fmt="%s")

100%|██████████| 50/50 [00:44<00:00,  1.11it/s]


#### Sim-PBMC 2

In [12]:
# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
# rare_type = random.sample(list(rare_name),2)
# ordinary_type = random.sample(list(ordinary_name),1)
rare_type = ['HSC','Plasma cell']
ordinary_type = ['CD4+ T naive']
type_choice = ordinary_type + rare_type
total_num = 3
filetype_name = 'Ordinary1_Rare2'

for file_num in tqdm(range(50)):
    name_list =type_choice
    l = list()
    num_list = list()

    for i in range(0,total_num):
        if total_num == 2:
            if i == 0:
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),490)
                num_list = num_list+['490']
            else :
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),10)
                num_list = num_list+['10']
        else:
            if i == 0:
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),480)
                num_list = num_list+['480']
            else:
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),10)
                num_list = num_list+['10']
    l = np.sort(l)
    label = true_label[l]
    # file_name
    file_name = ''
    for i in range(total_num):
        if i <total_num-1:
            file_name = file_name +str(name_list[i])+str(num_list[i])+'_'
        else:
            file_name = file_name +str(name_list[i])+str(num_list[i])

    Gene_Cell, Peak_Cell, Gene_Peak, gene_index, peak_index, cell_index = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    GP_matrix = Gene_Peak.X
    
    sub_gene_name = list(gene_names[0][gene_index])
    sub_peak_name = list(peak_names[0][peak_index])
    
    # python_input
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA_'+file_name+'.txt', np.array([]))
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/label500.npy',label)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA'+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/ATAC'+'.npz', ATAC_matrix)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/Gene_Peak_'+'.npz', GP_matrix)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/gene_name.npy',sub_gene_name)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/peak_name.npy',sub_peak_name)
    
    ## R_input
    site1_GEX = np.array(RNA_matrix.transpose().todense(),dtype='int64')
    site1_ATAC = np.array(ATAC_matrix.transpose().todense(),dtype='int64')
    site1_label = label
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_GEX.txt',site1_GEX,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_ATAC.txt',site1_ATAC,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_label.txt',site1_label,fmt="%s")

100%|██████████| 50/50 [00:44<00:00,  1.13it/s]


#### Sim-PBMC 3

In [15]:
# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
# ordinary_type = random.sample(list(ordinary_name),2)
ordinary_num = 2
total_num = 2
type_choice = ['CD8+ T','Erythroblast']
filetype_name = 'Ordinary2'

for file_num in tqdm(range(50)):
    name_list =type_choice
    l = list()
    num_list = list()

    for i in ['CD8+ T','Erythroblast']:
        if (i == 'CD8+ T'):
            l = l + random.sample(set(true_label[true_label==i].index),490)
            num_list = num_list+['490']
        elif (i == 'Erythroblast'):
            l = l + random.sample(set(true_label[true_label==i].index),10) 
            num_list = num_list+['10']
    l = np.sort(l)
    
    label = true_label[l]
    # file_name
    file_name = ''
    for i in range(total_num):
        if i <total_num-1:
            file_name = file_name +str(name_list[i])+str(num_list[i])+'_'
        else:
            file_name = file_name +str(name_list[i])+str(num_list[i])

    Gene_Cell, Peak_Cell, Gene_Peak, gene_index, peak_index, cell_index = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    GP_matrix = Gene_Peak.X
    
    sub_gene_name = list(gene_names[0][gene_index])
    sub_peak_name = list(peak_names[0][peak_index])
    
    ## python_input
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA_'+file_name+'.txt', np.array([]))
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/label500.npy',label)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA'+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/ATAC'+'.npz', ATAC_matrix)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/Gene_Peak_'+'.npz', GP_matrix)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/gene_name.npy',sub_gene_name)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/peak_name.npy',sub_peak_name)
    
    ## R_input
    site1_GEX = np.array(RNA_matrix.transpose().todense(),dtype='int64')
    site1_ATAC = np.array(ATAC_matrix.transpose().todense(),dtype='int64')
    site1_label = label
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_GEX.txt',site1_GEX,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_ATAC.txt',site1_ATAC,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_label.txt',site1_label,fmt="%s")

100%|██████████| 50/50 [00:43<00:00,  1.14it/s]


#### Sim-PBMC 4

In [16]:
# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
# ordinary_type = random.sample(list(ordinary_name),2)
ordinary_num = 2
total_num = 3
type_choice = ['CD8+ T','Erythroblast','HSC']
filetype_name = 'Ordinary2_Rare1'

for file_num in tqdm(range(50)):
    name_list = type_choice
    l = list()
    num_list = list()

    for i in ['CD8+ T','Erythroblast','HSC']:
        if (i == 'CD8+ T'):
            l = l + random.sample(set(true_label[true_label==i].index),480)
            num_list = num_list+['480']
        elif (i == 'HSC'):
            l = l + random.sample(set(true_label[true_label==i].index),10)
            num_list = num_list+['10']
        else:
            l = l + random.sample(set(true_label[true_label==i].index),10)
            num_list = num_list+['10']
    l = np.sort(l)
    
    label = true_label[l]
    # file_name
    file_name = ''
    for i in range(total_num):
        if i <total_num-1:
            file_name = file_name +str(name_list[i])+str(num_list[i])+'_'
        else:
            file_name = file_name +str(name_list[i])+str(num_list[i])

    Gene_Cell, Peak_Cell, Gene_Peak, gene_index, peak_index, cell_index = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    GP_matrix = Gene_Peak.X
    
    sub_gene_name = list(gene_names[0][gene_index])
    sub_peak_name = list(peak_names[0][peak_index])
    
    ## python_input
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA_'+file_name+'.txt', np.array([]))
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/label500.npy',label)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA'+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/ATAC'+'.npz', ATAC_matrix)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/Gene_Peak_'+'.npz', GP_matrix)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/gene_name.npy',sub_gene_name)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/peak_name.npy',sub_peak_name)
    
    ## R_input
    site1_GEX = np.array(RNA_matrix.transpose().todense(),dtype='int64')
    site1_ATAC = np.array(ATAC_matrix.transpose().todense(),dtype='int64')
    site1_label = label
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_GEX.txt',site1_GEX,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_ATAC.txt',site1_ATAC,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_label.txt',site1_label,fmt="%s")

100%|██████████| 50/50 [00:44<00:00,  1.13it/s]


#### Sim-PBMC 5

In [17]:
# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
# ordinary_type = random.sample(list(ordinary_name),2)
ordinary_num = 3
total_num = 3
type_choice = ['CD8+ T','Erythroblast','Naive CD20+ B']
filetype_name = 'Ordinary3'

for file_num in tqdm(range(50)):
    name_list =type_choice
    l = list()
    num_list = list()

    for i in ['CD8+ T','Erythroblast','Naive CD20+ B']:
        if (i == 'CD8+ T'):
            l = l + random.sample(set(true_label[true_label==i].index),480)
            num_list = num_list+['480']
        elif (i == 'Erythroblast'):
            l = l + random.sample(set(true_label[true_label==i].index),10)
            num_list = num_list+['10']
        else:
            l = l + random.sample(set(true_label[true_label==i].index),10)
            num_list = num_list+['10']
    l = np.sort(l)
    
    label = true_label[l]
    # file_name
    file_name = ''
    for i in range(total_num):
        if i <total_num-1:
            file_name = file_name +str(name_list[i])+str(num_list[i])+'_'
        else:
            file_name = file_name +str(name_list[i])+str(num_list[i])

    Gene_Cell, Peak_Cell, Gene_Peak, gene_index, peak_index, cell_index = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    GP_matrix = Gene_Peak.X
    
    sub_gene_name = list(gene_names[0][gene_index])
    sub_peak_name = list(peak_names[0][peak_index])
    
    ## python_input
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA_'+file_name+'.txt', np.array([]))
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/label500.npy',label)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/RNA'+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/ATAC'+'.npz', ATAC_matrix)
    sparse.save_npz(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/Gene_Peak_'+'.npz', GP_matrix)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/gene_name.npy',sub_gene_name)
    np.save(ip_file+str(filetype_name)+'/'+str(file_num)+'/P_input'+'/peak_name.npy',sub_peak_name)
    
    ## R_input
    site1_GEX = np.array(RNA_matrix.transpose().todense(),dtype='int64')
    site1_ATAC = np.array(ATAC_matrix.transpose().todense(),dtype='int64')
    site1_label = label
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_GEX.txt',site1_GEX,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_ATAC.txt',site1_ATAC,fmt="%d")
    np.savetxt(ip_file+str(filetype_name)+'/'+str(file_num)+'/R_input/'+'site1_label.txt',site1_label,fmt="%s")

100%|██████████| 50/50 [00:44<00:00,  1.13it/s]


#### Sim-PBMC 6

In [21]:
def sub_anndatas(index,ad1,dim1,ad2,dim2,ad3):
    def sub_ad(ad,id,dim):
        if dim == 0:
            ad = ad[id,:]
        elif dim == 1:
            ad = ad[:,id]
        return(ad)

    # sub sample
    ad1 = sub_ad(ad1,index,dim1)
    ad2 = sub_ad(ad2,index,dim2)

    # delete feature1 and feature2 not in sample
    index1 = np.where(np.array(ad1.X.sum(dim1)).squeeze()>0)[0]
    index2 = np.where(np.array(ad2.X.sum(dim2)).squeeze()>0)[0]
    ad1 = sub_ad(ad1,index1,1-dim1)
    ad2 = sub_ad(ad2,index2,1-dim2)
    ad3 = sub_ad(sub_ad(ad3,index1,0),index2,1)

    # delet feature1 and feature2 not link
    index1 = np.where(np.array(ad3.X.sum(1)).squeeze()>0)[0]
    index2 = np.where(np.array(ad3.X.sum(0)).squeeze()>0)[0]
    ad1 = sub_ad(ad1,index1,1-dim1)
    ad2 = sub_ad(ad2,index2,1-dim2)
    ad3 = sub_ad(sub_ad(ad3,index1,0),index2,1)

    #delet sample don't in features
    index = np.where((np.array(ad1.X.sum(1-dim1)).squeeze()>0) & (np.array(ad2.X.sum(1-dim2)).squeeze()>0))[0]
    ad1 = sub_ad(ad1,index,dim1)
    ad2 = sub_ad(ad2,index,dim2)
    return (ad1,ad2,ad3)

In [22]:
# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
# ordinary_type = random.sample(list(ordinary_name),2)
ordinary_num = 2
total_num = 2
type_choice = ['CD14+ Mono','CD8+ T']

for file_num in tqdm(range(50)):
    name_list =type_choice
    l = list()
    num_list = list()

    for i in range(0,total_num):
        if total_num == 2:
            if i == 0:
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),250)
                num_list = num_list+['250']
            else :
                l = l + random.sample(set(true_label[true_label==type_choice[i]].index),250)
                num_list = num_list+['250']

    # file_name
    file_name = ''
    for i in range(total_num):
        if i <total_num-1:
            file_name = file_name +str(name_list[i])+str(num_list[i])+'_'
        else:
            file_name = file_name +str(name_list[i])+str(num_list[i])

    label = true_label[l]
    Gene_Cell, Peak_Cell, Gene_Peak = sub_anndatas(l,gene_cell,1,peak_cell,1,gene_peak)
    #gene cell
    RNA_matrix = Gene_Cell.X
    #peak cell
    ATAC_matrix = Peak_Cell.X
    #gene peak
    RP_matrix = Gene_Peak.X

    os.mkdir(str(file_num))
    
    np.save(ip_file+str(file_num)+'/label500.npy',label)
    sparse.save_npz(ip_file+str(file_num)+'/RNA_'+file_name+'.npz', RNA_matrix) 
    sparse.save_npz(ip_file+str(file_num)+'/ATAC_'+file_name+'.npz', ATAC_matrix) 

100%|██████████| 50/50 [00:44<00:00,  1.12it/s]


#### Sim-PBMC 7-9

In [38]:
os.chdir('/fs/ess/PCON0022/dmt/data/dataset/data1/')
cell_type = pd.read_csv('cell_type.csv', index_col=0)

gene_peak = anndata.read_mtx('Gene_Peak.mtx')
gene_cell = anndata.read_mtx('Gene_Cell.mtx')
peak_cell = anndata.read_mtx('Peak_Cell.mtx')
gene_names = pd.read_csv('Gene_names.tsv', sep='\t', header=None)
cell_names = pd.read_csv('Cell_names.tsv', sep='\t', header=None)
peak_names = pd.read_csv('Peak_names.tsv', sep='\t', header=None)
peak_cell.obs_names = peak_names[0]
peak_cell.var_names = cell_names[0]
gene_cell.obs_names = gene_names[0]
gene_cell.var_names = cell_names[0]
gene_peak.obs_names = gene_names[0]
gene_peak.var_names = peak_names[0]

In [39]:
#Gene_Cell, Peak_Cell, Gene_Peak = gene_cell,peak_cell,gene_peak
true_label = cell_type.loc[cell_names[0].tolist(), :]
cell_type['number'] = 1
cell_count = cell_type.groupby('cell_type').count()
cell_count =cell_count.sort_values(by='number',ascending=True, inplace=False)

In [40]:
cell_count['percentage'] = (cell_count['number'] / cell_count['number'].sum())
cell_count

,number,percentage
cell_type,,
ID2-hi myeloid prog,108,0.001560
Plasma cell,379,0.005473
ILC,835,0.012058
cDC2,859,0.012405
MK/E prog,884,0.012766
CD8+ T naive,1012,0.014614
HSC,1072,0.015480
pDC,1191,0.017199
G/M prog,1203,0.017372


In [41]:
# Reset index
cell_df = true_label.reset_index(drop=True)

# Group by cell type to get the corresponding index list
cell_type_dict = cell_df.groupby('cell_type').apply(lambda x: x.index.tolist()).to_dict()

# Sort the dictionary based on the number of cells in each type
sorted_cell_type_dict = {k: v for k, v in sorted(cell_type_dict.items(), key=lambda item: len(item[1]))}

# Get the lengths of each cell type's indices
lengths = [len(indices) for indices in sorted_cell_type_dict.values()]

# Calculate the total count
total = sum(lengths)

# Print the proportion of each cell type
for cell_type, indices in sorted_cell_type_dict.items():
    proportion = len(indices) / total
    print(f"Cell type: {cell_type}, Proportion: {proportion:.3f}")
    
# Uncomment the lines below to print the keys and lengths of the sorted dictionary
# print(sorted_cell_type_dict.keys())
# for cell_type, indices in sorted_cell_type_dict.items():
#     print(f"Cell type: {cell_type}, Length: {len(indices)}")

Cell type: Plasma cell, Proportion: 0.004
Cell type: HSC, Proportion: 0.006
Cell type: ID2-hi myeloid prog, Proportion: 0.006
Cell type: G/M prog, Proportion: 0.008
Cell type: Lymph prog, Proportion: 0.008
Cell type: MK/E prog, Proportion: 0.009
Cell type: Transitional B, Proportion: 0.010
Cell type: pDC, Proportion: 0.010
Cell type: CD8+ T naive, Proportion: 0.012
Cell type: cDC2, Proportion: 0.012
Cell type: Proerythroblast, Proportion: 0.012
Cell type: CD16+ Mono, Proportion: 0.016
Cell type: B1 B, Proportion: 0.023
Cell type: Normoblast, Proportion: 0.032
Cell type: ILC, Proportion: 0.034
Cell type: Naive CD20+ B, Proportion: 0.037
Cell type: Erythroblast, Proportion: 0.076
Cell type: CD14+ Mono, Proportion: 0.108
Cell type: CD4+ T activated, Proportion: 0.112
Cell type: NK, Proportion: 0.132
Cell type: CD4+ T naive, Proportion: 0.137
Cell type: CD8+ T, Proportion: 0.196


#### Sim-PBMC 7

In [ ]:
import numpy as np
import random

def sample_cells(cell_type_dict, total_samples=5000):
    # Extract all cell types
    cell_types = list(cell_type_dict.keys())
    
    # The last 4 types must be sampled
    fixed_types = cell_types[-4:]
    
    # Randomly select 1 cell type from the first types
    random_type = random.choice(cell_types[:2])
    
    # Combine these 5 types
    sample_types = fixed_types + [random_type]
    
    # Calculate the proportion of each cell type within the selected types
    type_counts = {cell_type: len(cells) for cell_type, cells in cell_type_dict.items() if cell_type in sample_types}
    total_count = sum(type_counts.values()) - 50 if random_type in type_counts else 0
    type_ratio = {cell_type: count / total_count for cell_type, count in type_counts.items() if cell_type != random_type}
    
    # Sample these cell types proportionally to a total of 5000
    sample_counts = {cell_type: int(np.round(ratio * total_samples)) for cell_type, ratio in type_ratio.items()}
    
    # Find the difference between the total number of samples and 5000 (including the fixed 50), then allocate this difference to the cell type with the largest sample count
    diff = total_samples - sum(sample_counts.values()) - 50
    max_type = max(sample_counts, key=sample_counts.get)
    sample_counts[max_type] += diff
    
    # Perform sampling
    sample_indices = []
    for cell_type, count in sample_counts.items():
        cells = cell_type_dict[cell_type]
        if len(cells) > count:
            cells = np.random.choice(cells, size=count, replace=False)
        sample_indices.extend(cells)
    
    # Additionally sample 50 samples from the selected random type
    cells = cell_type_dict[random_type]
    if len(cells) > 50:
        cells = np.random.choice(cells, size=50, replace=False)
    sample_indices.extend(cells)
        
    return sample_indices

In [29]:
import os
import numpy as np
import scipy.sparse
from tqdm import tqdm
from scipy import io

# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
filetype_name_list = ['Rare1_Ordinary4']

for filetype_name in filetype_name_list:
    for file_num in tqdm(range(0, 50)):  # Process the first 50 folders
        path2 = os.path.join(ip_file, filetype_name, str(file_num), 'R_input')
        
        # Set random seed
        np.random.seed(file_num)
        random.seed(file_num)
        
        # Execute sampling function
        sample_indices = sample_cells(sorted_cell_type_dict, total_samples=5000)
        
        # Sample corresponding gene_cell, peak_cell matrices, and true_label labels
        sub_gene_cell = gene_cell[:, sample_indices]
        sub_peak_cell = peak_cell[:, sample_indices]
        gene_cell_matrix = sub_gene_cell.X
        peak_cell_matrix = sub_peak_cell.X
        
        sub_label = list(true_label.iloc[sample_indices]['cell_type'])
        
        print('sub_gene_cell.shape:', end=' ')
        print(sub_gene_cell.shape)
        print('sub_peak_cell.shape:', end=' ')
        print(sub_peak_cell.shape)
        
        # Save files
        io.mmwrite(os.path.join(path2, "sub_gene_cell.mtx"), gene_cell_matrix)
        io.mmwrite(os.path.join(path2, "sub_peak_cell.mtx"), peak_cell_matrix)
        with open(os.path.join(path2, "sub_label.txt"), "w") as file:
            for label in sub_label:
                file.write(f"{label}\n")
        with open(os.path.join(path2, "sample_indices.txt"), "w") as file:
            for index in sample_indices:
                file.write(f"{index}\n")

  2%|▏         | 1/50 [00:00<00:24,  2.01it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


  4%|▍         | 2/50 [00:00<00:21,  2.20it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


  6%|▌         | 3/50 [00:01<00:20,  2.28it/s]

...

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


 98%|█████████▊| 49/50 [00:20<00:00,  2.39it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


100%|██████████| 50/50 [00:20<00:00,  2.39it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


#### Sim-PBMC 8

In [ ]:
import numpy as np
import random

def sample_cells(cell_type_dict, total_samples=5000):
    # Extract all cell types
    cell_types = list(cell_type_dict.keys())
    
    # The last 9 types must be sampled
    fixed_types = cell_types[-9:]
    
    # Randomly select 1 cell type from the first types
    random_type = random.choice(cell_types[:2])
    
    # Combine these 10 types
    sample_types = fixed_types + [random_type]
    
    # Calculate the proportion of each cell type within the selected types
    type_counts = {cell_type: len(cells) for cell_type, cells in cell_type_dict.items() if cell_type in sample_types}
    total_count = sum(type_counts.values()) - 50 if random_type in type_counts else 0
    type_ratio = {cell_type: count / total_count for cell_type, count in type_counts.items() if cell_type != random_type}
    
    # Sample these cell types proportionally to a total of 5000
    sample_counts = {cell_type: int(np.round(ratio * total_samples)) for cell_type, ratio in type_ratio.items()}
    
    # Find the difference between the total number of samples and 5000 (including the fixed 50), then allocate this difference to the cell type with the largest sample count
    diff = total_samples - sum(sample_counts.values()) - 50
    max_type = max(sample_counts, key=sample_counts.get)
    sample_counts[max_type] += diff
    
    # Perform sampling
    sample_indices = []
    for cell_type, count in sample_counts.items():
        cells = cell_type_dict[cell_type]
        if len(cells) > count:
            cells = np.random.choice(cells, size=count, replace=False)
        sample_indices.extend(cells)
    
    # Additionally sample 50 samples from the selected random type
    cells = cell_type_dict[random_type]
    if len(cells) > 50:
        cells = np.random.choice(cells, size=50, replace=False)
    sample_indices.extend(cells)
        
    return sample_indices


In [31]:
import os
import numpy as np
import scipy.sparse
from tqdm import tqdm
from scipy import io

# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
filetype_name_list = ['Rare1_Ordinary9']

for filetype_name in filetype_name_list:
    for file_num in tqdm(range(0, 50)):  # Process the first 50 folders
        path2 = os.path.join(ip_file, filetype_name, str(file_num), 'R_input')
        
        # Set random seed
        np.random.seed(file_num)
        random.seed(file_num)
        
        # Execute sampling function
        sample_indices = sample_cells(sorted_cell_type_dict, total_samples=5000)
        
        # Sample corresponding gene_cell, peak_cell matrices, and true_label labels
        sub_gene_cell = gene_cell[:, sample_indices]
        sub_peak_cell = peak_cell[:, sample_indices]
        gene_cell_matrix = sub_gene_cell.X
        peak_cell_matrix = sub_peak_cell.X
        
        sub_label = list(true_label.iloc[sample_indices]['cell_type'])
        
        print('sub_gene_cell.shape:', end=' ')
        print(sub_gene_cell.shape)
        print('sub_peak_cell.shape:', end=' ')
        print(sub_peak_cell.shape)
        
        # Save files
        io.mmwrite(os.path.join(path2, "sub_gene_cell.mtx"), gene_cell_matrix)
        io.mmwrite(os.path.join(path2, "sub_peak_cell.mtx"), peak_cell_matrix)
        with open(os.path.join(path2, "sub_label.txt"), "w") as file:
            for label in sub_label:
                file.write(f"{label}\n")
        with open(os.path.join(path2, "sample_indices.txt"), "w") as file:
            for index in sample_indices:
                file.write(f"{index}\n")


  2%|▏         | 1/50 [00:00<00:25,  1.95it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


  4%|▍         | 2/50 [00:00<00:22,  2.15it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


  6%|▌         | 3/50 [00:01<00:21,  2.22it/s]

...

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


 98%|█████████▊| 49/50 [00:20<00:00,  2.40it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


100%|██████████| 50/50 [00:21<00:00,  2.36it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


#### Sim-PBMC 9

In [ ]:
import numpy as np
import random

def sample_cells(cell_type_dict, total_samples=5000):
    # Extract all cell types
    cell_types = list(cell_type_dict.keys())
    
    # The last ten types must be sampled
    fixed_types = cell_types[-10:]
    
    # Randomly select 5 cell types from the first types
    random_types = random.sample(cell_types[:8], 5)
    
    # Combine these 15 types
    sample_types = fixed_types + random_types
    
    # Calculate the proportion of each cell type within these 15 types
    type_counts = {cell_type: len(cells) for cell_type, cells in cell_type_dict.items() if cell_type in sample_types}
    total_count = sum(type_counts.values())
    type_ratio = {cell_type: count / total_count for cell_type, count in type_counts.items()}
    
    # Sample these cell types proportionally to a total of 5000
    sample_counts = {cell_type: int(np.round(ratio * total_samples)) for cell_type, ratio in type_ratio.items()}
    
    # Find the difference between the total number of samples and 5000, then allocate this difference to the cell type with the largest sample count
    diff = total_samples - sum(sample_counts.values())
    max_type = max(sample_counts, key=sample_counts.get)
    sample_counts[max_type] += diff
    
    # Perform sampling
    sample_indices = []
    for cell_type, count in sample_counts.items():
        cells = cell_type_dict[cell_type]
        if len(cells) > count:
            cells = np.random.choice(cells, size=count, replace=False)
        sample_indices.extend(cells)
        
    return sample_indices

In [32]:
import os
import numpy as np
import scipy.sparse
from tqdm import tqdm
from scipy import io

# ip_file = '/fs/ess/scratch/PCON0022/mtduan/site1/'
filetype_name_list = ['Rare5_Ordinary10']

for filetype_name in filetype_name_list:
    for file_num in tqdm(range(0, 50)):  # Process 50 folders
        path2 = os.path.join(ip_file, filetype_name, str(file_num), 'R_input')
        
        # Set random seed
        np.random.seed(file_num)
        random.seed(file_num)
        
        # Execute sampling function
        sample_indices = sample_cells(sorted_cell_type_dict, total_samples=5000)
        
        # Sample corresponding gene_cell, peak_cell matrices, and true_label labels
        sub_gene_cell = gene_cell[:, sample_indices]
        sub_peak_cell = peak_cell[:, sample_indices]
        gene_cell_matrix = sub_gene_cell.X
        peak_cell_matrix = sub_peak_cell.X
        
        sub_label = list(true_label.iloc[sample_indices]['cell_type'])
        
        print('sub_gene_cell.shape:', end=' ')
        print(sub_gene_cell.shape)
        print('sub_peak_cell.shape:', end=' ')
        print(sub_peak_cell.shape)
        
        # Save files
        io.mmwrite(os.path.join(path2, "sub_gene_cell.mtx"), gene_cell_matrix)
        io.mmwrite(os.path.join(path2, "sub_peak_cell.mtx"), peak_cell_matrix)
        with open(os.path.join(path2, "sub_label.txt"), "w") as file:
            for label in sub_label:
                file.write(f"{label}\n")
        with open(os.path.join(path2, "sample_indices.txt"), "w") as file:
            for index in sample_indices:
                file.write(f"{index}\n")

  2%|▏         | 1/50 [00:00<00:24,  1.97it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


  4%|▍         | 2/50 [00:00<00:21,  2.21it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


  6%|▌         | 3/50 [00:01<00:20,  2.31it/s]

...

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


 98%|█████████▊| 49/50 [00:20<00:00,  2.38it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)


100%|██████████| 50/50 [00:20<00:00,  2.39it/s]

sub_gene_cell.shape: (12152, 5000)
sub_peak_cell.shape: (116490, 5000)
